In [1]:
pip install tensorflow-macos tensorflow-metal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 MB 7.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 10.6 MB/s eta 0:00:00 MB/s eta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml_dtypes 0.5.3
    Uninstalling ml_dtypes-0.5.3:
      Successfully uninstalled ml_dtypes-0.5.3
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.20.0
    Uninstalling tensorboard-2.20.0:
      Successfully uninstalled tensorboard-2.20.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.20.0
    Uninstalling tensorflow-2.20.0:
      Successfully uninstalled tensorflow-2.20.0
Note: you may need to restart the kernel to use updated 

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, GRU, SimpleRNN, Dense, Input, Bidirectional, LSTMCell, GRUCell, SimpleRNNCell
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

In [13]:
# Define hyperparameters and dataset-related values
vocab_size = 10000
embedding_dim = 128
lstm_units = 64
gru_units = 64
rnn_units = 64
num_epochs = 10
batch_size = 64

In [15]:
# Generate sample data (placeholders)
x_train = tf.random.uniform((1000, 50), minval=0, maxval=vocab_size, dtype=tf.int32)
y_train = tf.random.uniform((1000, 1), minval=0, maxval=2, dtype=tf.int32)
encoder_input_train = tf.random.uniform((1000, 20), minval=0, maxval=vocab_size, dtype=tf.int32)
decoder_input_train = tf.random.uniform((1000, 25), minval=0, maxval=vocab_size, dtype=tf.int32)
decoder_output_train = tf.random.uniform((1000, 25, vocab_size), minval=0, maxval=2, dtype=tf.float32)

In [17]:
# Create a NestedCell Custom Layer
class NestedCell(Layer):
   def __init__(self, units, **kwargs):
       super(NestedCell, self).__init__(**kwargs)
       self.units = units
       self.lstm_cell = LSTMCell(self.units)
       self.gru_cell = GRUCell(self.units)
       self.rnn_cell = SimpleRNNCell(self.units)
   def call(self, inputs, states):
       return self.lstm_cell(inputs, states), self.gru_cell(inputs, states), self.rnn_cell(inputs, states)

In [19]:
# Create instances of the different models
def create_embedding_lstm_model():
   input_layer = Input(shape=(None,))
   embedded = Embedding(vocab_size, embedding_dim)(input_layer)
   lstm_output = LSTM(lstm_units)(embedded)
   output_layer = Dense(1, activation='sigmoid')(lstm_output)
   model = Model(inputs=input_layer, outputs=output_layer)
   return model

In [21]:
# Create an Embedding + GRU + SimpleRNN + Dense model
def create_embedding_gru_rnn_model():
   input_layer = Input(shape=(None,))
   embedded = Embedding(vocab_size, embedding_dim)(input_layer)
   gru_output = GRU(gru_units, return_sequences=True)(embedded)
   rnn_output = SimpleRNN(rnn_units)(gru_output)
   output_layer = Dense(1, activation='sigmoid')(rnn_output)
   model = Model(inputs=input_layer, outputs=output_layer)
   return model


In [23]:
# Create an Encoder-Decoder model
def create_encoder_decoder_model():
   encoder_inputs = Input(shape=(None,))
   encoder_embedded = Embedding(vocab_size, embedding_dim)(encoder_inputs)
   encoder_lstm = LSTM(lstm_units, return_state=True)
   _, state_h, state_c = encoder_lstm(encoder_embedded)
   decoder_inputs = Input(shape=(None,))
   decoder_embedded = Embedding(vocab_size, embedding_dim)(decoder_inputs)
   decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
   decoder_outputs, _, _ = decoder_lstm(decoder_embedded, initial_state=[state_h, state_c])
   output_layer = Dense(vocab_size, activation='softmax')(decoder_outputs)
   model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=output_layer)
   return model

In [25]:
# Create a Bidirectional LSTM model
def create_bidirectional_lstm_model():
   input_layer = Input(shape=(None,))
   embedded = Embedding(vocab_size, embedding_dim)(input_layer)
   bidirectional_lstm = Bidirectional(LSTM(lstm_units))(embedded)
   output_layer = Dense(1, activation='sigmoid')(bidirectional_lstm)
   model = Model(inputs=input_layer, outputs=output_layer)
   return model

In [27]:
# Create a NestedCell Custom Layer
class NestedCell(Layer):
   def __init__(self, units, **kwargs):
       super(NestedCell, self).__init__(**kwargs)
       self.units = units
       self.lstm_cell = LSTMCell(self.units)
       self.gru_cell = GRUCell(self.units)
       self.rnn_cell = SimpleRNNCell(self.units)
   def call(self, inputs, states):
       return self.lstm_cell(inputs, states), self.gru_cell(inputs, states), self.rnn_cell(inputs, states)# Create instances of the different models
embedding_lstm_model = create_embedding_lstm_model()
embedding_gru_rnn_model = create_embedding_gru_rnn_model()
encoder_decoder_model = create_encoder_decoder_model()
bidirectional_lstm_model = create_bidirectional_lstm_model()


In [29]:
# Create instances of the different models
embedding_lstm_model = create_embedding_lstm_model()
embedding_gru_rnn_model = create_embedding_gru_rnn_model()
encoder_decoder_model = create_encoder_decoder_model()
bidirectional_lstm_model = create_bidirectional_lstm_model()

In [31]:
# Compile the models
embedding_lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
embedding_gru_rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
encoder_decoder_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
bidirectional_lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [33]:
embedding_lstm_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)
embedding_gru_rnn_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)
encoder_decoder_model.fit([encoder_input_train, decoder_input_train], decoder_output_train, epochs=num_epochs, batch_size=batch_size)
bidirectional_lstm_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)

Epoch 1/10


2025-11-04 07:41:50.667298: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.4980 - loss: 0.6938
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9590 - loss: 0.6458
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9540 - loss: 0.3462
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9950 - loss: 0.0437
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0091
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0040
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0026
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0018
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0014
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0011
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.5040 - loss: 0.6995
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.9810 - loss:

In [35]:
# Print model summaries
print("Embedding + LSTM Model:")
embedding_lstm_model.summary()

print("\nEmbedding + GRU + SimpleRNN Model:")
embedding_gru_rnn_model.summary()

print("\nEncoder-Decoder Model:")
encoder_decoder_model.summary()

print("\nBidirectional LSTM Model:")
bidirectional_lstm_model.summary()

Embedding + LSTM Model:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 128)      │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,988,421 (15.21 MB)

 Trainable params: 1,329,473 (5.07 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_4 (Embedding)         │ (None, None, 128)      │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,136,837 (15.78 MB)

 Trainable params: 1,378,945 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,757,892 (10.52 MB)

In [37]:
# Generate sample data (placeholders)
x_train = tf.random.uniform((1000, 50), minval=0, maxval=vocab_size, dtype=tf.int32)
y_train = tf.random.uniform((1000, 1), minval=0, maxval=2, dtype=tf.int32)
encoder_input_train = tf.random.uniform((1000, 20), minval=0, maxval=vocab_size, dtype=tf.int32)
decoder_input_train = tf.random.uniform((1000, 25), minval=0, maxval=vocab_size, dtype=tf.int32)
decoder_output_train = tf.random.uniform((1000, 25, vocab_size), minval=0, maxval=2, dtype=tf.float32)

In [39]:
# Train the models
embedding_lstm_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)
embedding_gru_rnn_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)
encoder_decoder_model.fit([encoder_input_train, decoder_input_train], decoder_output_train, epochs=num_epochs, batch_size=batch_size)
bidirectional_lstm_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5020 - loss: 1.1851
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8200 - loss: 0.5556
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9410 - loss: 0.4128
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9860 - loss: 0.2334
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9960 - loss: 0.0999
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0427
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0248
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0174
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0134
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0110
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.4840 - loss: 1.0398
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.7

In [41]:
# Evaluate the models on training data
print("\nEvaluating models...\n")
lstm_eval = embedding_lstm_model.evaluate(x_train, y_train, verbose=1)
gru_rnn_eval = embedding_gru_rnn_model.evaluate(x_train, y_train, verbose=1)
encoder_decoder_eval = encoder_decoder_model.evaluate([encoder_input_train, decoder_input_train], decoder_output_train, verbose=1)
bilstm_eval = bidirectional_lstm_model.evaluate(x_train, y_train, verbose=1)


Evaluating models...

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 1.0000 - loss: 0.0098
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 118ms/step - accuracy: 1.0000 - loss: 0.0023
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 8.0000e-05 - loss: 92168.4609
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 1.0000 - loss: 0.0096


In [43]:
# Print accuracy results
print("\nModel Performance Summary:")
print(f"Embedding + LSTM Model Accuracy: {lstm_eval[1]:.4f}")
print(f"Embedding + GRU + SimpleRNN Model Accuracy: {gru_rnn_eval[1]:.4f}")
print(f"Encoder-Decoder Model Accuracy: {encoder_decoder_eval[1]:.4f}")
print(f"Bidirectional LSTM Model Accuracy: {bilstm_eval[1]:.4f}")


Model Performance Summary:
Embedding + LSTM Model Accuracy: 1.0000
Embedding + GRU + SimpleRNN Model Accuracy: 1.0000
Encoder-Decoder Model Accuracy: 0.0001
Bidirectional LSTM Model Accuracy: 1.0000


In [45]:
# Generate predictions on sample data
print("\nGenerating sample predictions from Bidirectional LSTM Model:")
sample_input = tf.random.uniform((1, 50), minval=0, maxval=vocab_size, dtype=tf.int32)
prediction = bidirectional_lstm_model.predict(sample_input)
print("Sample Prediction Output:", prediction)


Generating sample predictions from Bidirectional LSTM Model:
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 693ms/step
Sample Prediction Output: [[0.9312979]]
